# On convergence rate bounds for a class of nonlinear Markov chains
*Aleksandr Shchegolev, Alexander Veretennikov*

In [1]:
from typing import List, Tuple
from itertools import product
from sympy import Symbol, Matrix, Min, Max, Add, Abs, simplify, zeros, Mul, Rational, N
from IPython.display import display

The function below takes transition probability matrix for a linear Markov chain $\bar P_x$ as an input and returns a tuple that contains a matrix for a corresponding coupled process $\bar V$, its eigenvalues and moduli of eigenvalues that are sorted in descending order.

In [2]:
def linear_coupled_process_spectral_radius(p: Matrix) -> Tuple[Matrix, List, List]:
    process_dim = p.rows
    coupled_ss = Mul(*p.shape) - process_dim

    print(f"The coupled process state space size is {coupled_ss}")

    prs = [list(x) for x in product(range(process_dim), repeat=2) if x[0] != x[1]]
    print(f"\nPairs:\n{prs}")
    coupl_proc = Matrix(zeros(*(coupled_ss, coupled_ss)))

    alphas = []
    for i in range(coupled_ss):
        for j in range(coupled_ss):
            alpha = Rational('0')
            for k in range(process_dim):
                alpha += Min(*(p[prs[i][0], k], p[prs[i][1], k]))
            alphas.append(alpha)
            coupl_proc[i, j] = ((p[prs[i][0], prs[j][0]] - Min(*(p[prs[i][0], prs[j][0]], p[prs[i][1], prs[j][0]]))) *
                                (p[prs[i][1], prs[j][1]] - Min(
                                    *(p[prs[i][1], prs[j][1]], p[prs[i][0], prs[j][1]])))) / (1 - Rational(f'{alpha}'))
    print('\nMarkov-Dobrushin 1-alpha:')
    display(1 - min(alphas))
    print("\nCoupled process:")
    display(coupl_proc)

    eigenvalues = list()
    for x, y in coupl_proc.eigenvals().items():
        for i in range(y):
            eigenvalues.append(x)
    eigenvalues = sorted(eigenvalues)[::-1]

    print("\nEigen values:")
    display(eigenvalues)

    eigen_modulus = sorted([Abs(val) for val in eigenvalues])[::-1]
    print("\nEigen values modulus:")
    display(eigen_modulus)
    return coupl_proc, eigenvalues, eigen_modulus

The second function takes transition probability matrix for a linear Markov chain $\bar P_x$ as an input and returns the estimate $1-\bar\alpha$.

In [3]:
def linear_markov_dobrushin_one_minus_alpha(p: Matrix) -> Tuple[Matrix, List, List]:
    process_dim = p.rows
    prs = [list(x) for x in product(range(process_dim), repeat=2) if x[0] != x[1]]

    alphas = []
    for i in range(len(prs)):
        alpha = Rational('0')
        for k in range(process_dim):
            alpha += Min(*(p[prs[i][0], k], p[prs[i][1], k]))
        alphas.append(alpha)
    return 1 - min(alphas)

## Example 1

Let us consider a discrete nMC with the state space $(E,\mathcal{E}) = \left(\{1, 2, 3, 4\}, 2^{\{1, 2, 3, 4\}}\right)$, the initial distribution $\mu_0 = (\mu^0_1, \mu^0_2, \mu^0_3, \mu^0_4)$ and transition probability matrix:
$$P_{\mu,x} = 
\begin{pmatrix}
0.4 - \kappa \mu_{1} & 0.2 & 0.2 + \kappa \mu_{1} & 0.2\\
0.3 & 0.4 & 0.2 & 0.1\\
0.2 & 0.2 & 0.4 & 0.2\\
0.2 & 0.1 & 0.2 & 0.5
\end{pmatrix},\quad
\bar P_x = 
\begin{pmatrix}
0.4 & 0.2 & 0.2 & 0.2\\
0.3 & 0.4 & 0.2 & 0.1\\
0.2 & 0.2 & 0.4 & 0.2\\
0.2 & 0.1 & 0.2 & 0.5
\end{pmatrix},
$$
where $0 \le \kappa < 0.3$.

In [4]:
# Defining symbols
kappa = Symbol('kappa')
mu1, mu2, mu3, mu4 = Symbol('mu_1'), Symbol('mu_2'), Symbol('mu_3'), Symbol('mu_4')

In [5]:
# Defining nMC transition matrix
p = Matrix([[Rational('0.4') - kappa * mu1, Rational('0.2'), Rational('0.2') + kappa * mu1, Rational('0.2')],
            [Rational('0.3'), Rational('0.4'), Rational('0.2'), Rational('0.1')],
            [Rational('0.2'), Rational('0.2'), Rational('0.4'), Rational('0.2')],
            [Rational('0.2'), Rational('0.1'), Rational('0.2'), Rational('0.5')]])
p

Matrix([
[-kappa*mu_1 + 2/5,  1/5, kappa*mu_1 + 1/5,  1/5],
[             3/10,  2/5,              1/5, 1/10],
[              1/5,  1/5,              2/5,  1/5],
[              1/5, 1/10,              1/5,  1/2]])

In [6]:
# Defining linear MC transition matrix
p_lin = Matrix([[Rational('0.4'), Rational('0.2'), Rational('0.2'), Rational('0.2')],
                [Rational('0.3'), Rational('0.4'), Rational('0.2'), Rational('0.1')],
                [Rational('0.2'), Rational('0.2'), Rational('0.4'), Rational('0.2')],
                [Rational('0.2'), Rational('0.1'), Rational('0.2'), Rational('0.5')]])
p_lin

Matrix([
[ 2/5,  1/5, 1/5,  1/5],
[3/10,  2/5, 1/5, 1/10],
[ 1/5,  1/5, 2/5,  1/5],
[ 1/5, 1/10, 1/5,  1/2]])

In [7]:
print(f'All row sum equals to 1 for nMC: {all([Add(*p.row(row_num)) == 1 for row_num in range(p.rows)])}')
print(f'All row sum equals to 1 for MC: {all([Add(*p_lin.row(row_num)) == 1 for row_num in range(p_lin.rows)])}')

All row sum equals to 1 for nMC: True
All row sum equals to 1 for MC: True


In [8]:
coupl_proc, eigenvalues, eigen_modulus = linear_coupled_process_spectral_radius(p_lin)

The coupled process state space size is 12

Pairs:
[[0, 1], [0, 2], [0, 3], [1, 0], [1, 2], [1, 3], [2, 0], [2, 1], [2, 3], [3, 0], [3, 1], [3, 2]]

Markov-Dobrushin 1-alpha:


2/5


Coupled process:


Matrix([
[1/10,    0,    0,    0,    0,    0,    0,    0,   0,    0, 1/10,   0],
[   0,  1/5,    0,    0,    0,    0,    0,    0,   0,    0,    0,   0],
[   0,    0,  1/5,    0,    0, 1/10,    0,    0,   0,    0,    0,   0],
[   0,    0,    0, 1/10,    0, 1/10,    0,    0,   0,    0,    0,   0],
[   0, 1/15, 1/30,    0, 2/15, 1/15,    0,    0,   0,    0,    0,   0],
[   0,    0, 1/10,    0,    0, 3/10,    0,    0,   0,    0,    0,   0],
[   0,    0,    0,    0,    0,    0,  1/5,    0,   0,    0,    0,   0],
[   0,    0,    0,    0,    0,    0, 1/15, 2/15,   0, 1/30, 1/15,   0],
[   0,    0,    0,    0,    0, 1/10,    0,    0, 1/5,    0,    0,   0],
[   0,    0,    0,    0,    0,    0,    0,    0,   0,  1/5, 1/10,   0],
[   0,    0,    0,    0,    0,    0,    0,    0,   0, 1/10, 3/10,   0],
[   0,    0,    0,    0,    0,    0,    0,    0,   0,    0, 1/10, 1/5]])


Eigen values:


[sqrt(5)/20 + 1/4,
 sqrt(5)/20 + 1/4,
 1/5,
 1/5,
 1/5,
 1/5,
 1/4 - sqrt(5)/20,
 1/4 - sqrt(5)/20,
 2/15,
 2/15,
 1/10,
 1/10]


Eigen values modulus:


[sqrt(5)/20 + 1/4,
 sqrt(5)/20 + 1/4,
 1/5,
 1/5,
 1/5,
 1/5,
 1/4 - sqrt(5)/20,
 1/4 - sqrt(5)/20,
 2/15,
 2/15,
 1/10,
 1/10]

Spectral radius $r$ is equal to the greatest eigenvalue modulus.

In [9]:
r = [N(eig_mod) for eig_mod in eigen_modulus][0]
r

0.361803398874989

In [10]:
linear_markov_dobrushin_one_minus_alpha(p_lin)

2/5

Let us calculate the values $1-\alpha_2, 1-\alpha_3, 1-\alpha_4, 1-\alpha_5$ compare them with $r^2, r^3, r^4, r^5$ respectively.

In [11]:
[N(linear_markov_dobrushin_one_minus_alpha(p_lin ** y)) for y in range(2,6)]

[0.150000000000000,
 0.0550000000000000,
 0.0200000000000000,
 0.00725000000000000]

In [12]:
[r ** y for y in range(2,6)]

[0.130901699437495,
 0.0473606797749979,
 0.0171352549156242,
 0.00619959346906221]

Therefore, $r = \frac{\sqrt{5}}{20} + \frac{1}{4} \approx 0.3618$, $r^2 \approx 0.1309$, $r^3 \approx 0.04736$, $r^4 \approx 0.017135$ , $r^5 \approx 0.0061996$, $1- \bar\alpha = 0.4$, $1- \bar\alpha_2 = 0.15$, $1 - \bar\alpha_3 = 0.055$, $1 - \bar\alpha_4 = 0.02$, $1 - \bar\alpha_5 = 0.00725$.
We have $r < 1 - \bar\alpha = 1 - \alpha$, $r^2 < 1-\bar\alpha_2$, $r^3 < 1-\bar\alpha_3$, $r^4 < 1-\bar\alpha_4$, $r^5 < 1-\bar\alpha_5$. This shows that the new assumption based on the spectral radius approach for linear MC and on nonlinear small perturbations is weaker that each of the MD conditions, at least, for $k=1, \ldots, 5$.

## Example 2

Let us consider another discrete nMC with a bit more involved nonlinear components. Let the nMC have the state space $(E,\mathcal{E}) = \left(\{1, 2, 3, 4, 5, 6\}, 2^{\{1, 2, 3, 4, 5, 6\}}\right)$, the initial distribution $\mu_0 = (\mu^0_1, \mu^0_2, \mu^0_3, \mu^0_4, \mu^0_5, \mu^0_6)$ and transition probability matrix:

$$P_{\mu,x} = 
\begin{pmatrix}
0.4 - \kappa\mu_{1}& 0.2 + \kappa\mu_{1} & 0.1 & 0.1 & 0.1 & 0.1\\
0.2 & 0.3 - \kappa\mu_{2}  & 0.2 + \kappa\mu_{2} & 0.1 & 0.1 & 0.1\\
0.1 & 0.2 & 0.3 - \kappa\mu_{3} & 0.2 + \kappa\mu_{3} & 0.1 & 0.1\\
0.1 & 0.1 & 0.2 & 0.3 - \kappa\mu_{4} & 0.2 + \kappa\mu_{4} & 0.1\\
0.1 & 0.1 & 0.1 & 0.2 & 0.3 - \kappa\mu_{5} & 0.2 + \kappa\mu_{5}\\
0.1 & 0.1 & 0.1 & 0.1 & 0.2 + \kappa\mu_{6} & 0.4 - \kappa\mu_{6}
\end{pmatrix},$$

where $0 \le \kappa < 0.3$.

The transition probability matrix for the corresponding linear Markov chain is
$$
\bar P_x = 
\begin{pmatrix}
0.4 & 0.2 & 0.1 & 0.1 & 0.1 & 0.1\\
0.2 & 0.3 & 0.2 & 0.1 & 0.1 & 0.1\\
0.1 & 0.2 & 0.3 & 0.2 & 0.1 & 0.1\\
0.1 & 0.1 & 0.2 & 0.3 & 0.2 & 0.1\\
0.1 & 0.1 & 0.1 & 0.2 & 0.3 & 0.2\\
0.1 & 0.1 & 0.1 & 0.1 & 0.2 & 0.4
\end{pmatrix}.
$$

In [13]:
# Defining symbols
kappa = Symbol('kappa')
mu1, mu2, mu3, mu4, mu5, mu6 = Symbol('mu_1'), Symbol('mu_2'), Symbol('mu_3'), Symbol('mu_4'), Symbol('mu_5'), Symbol('mu_6')

In [14]:
# Defining nMC transition matrix
p = Matrix([[Rational('0.4') - kappa * mu1, Rational('0.2') + kappa * mu1, Rational('0.1'), Rational('0.1'), Rational('0.1'), Rational('0.1')],
            [Rational('0.2'), Rational('0.3')- kappa * mu2, Rational('0.2') + kappa * mu2, Rational('0.1'), Rational('0.1'), Rational('0.1')],
            [Rational('0.1'), Rational('0.2'), Rational('0.3')- kappa * mu3, Rational('0.2') + kappa * mu3, Rational('0.1'), Rational('0.1')],
            [Rational('0.1'), Rational('0.1'), Rational('0.2'), Rational('0.3') - kappa * mu4, Rational('0.2') + kappa * mu4, Rational('0.1')],
            [Rational('0.1'), Rational('0.1'), Rational('0.1'), Rational('0.2'), Rational('0.3') - kappa * mu5, Rational('0.2') + kappa * mu5],
            [Rational('0.1'), Rational('0.1'), Rational('0.1'), Rational('0.1'), Rational('0.2') + kappa * mu6, Rational('0.4') - kappa * mu6]])
p

Matrix([
[-kappa*mu_1 + 2/5,   kappa*mu_1 + 1/5,               1/10,               1/10,               1/10,              1/10],
[              1/5, -kappa*mu_2 + 3/10,   kappa*mu_2 + 1/5,               1/10,               1/10,              1/10],
[             1/10,                1/5, -kappa*mu_3 + 3/10,   kappa*mu_3 + 1/5,               1/10,              1/10],
[             1/10,               1/10,                1/5, -kappa*mu_4 + 3/10,   kappa*mu_4 + 1/5,              1/10],
[             1/10,               1/10,               1/10,                1/5, -kappa*mu_5 + 3/10,  kappa*mu_5 + 1/5],
[             1/10,               1/10,               1/10,               1/10,   kappa*mu_6 + 1/5, -kappa*mu_6 + 2/5]])

In [15]:
# Defining linear MC transition matrix
p_lin = Matrix([[Rational('0.4'), Rational('0.2'), Rational('0.1'), Rational('0.1'), Rational('0.1'), Rational('0.1')],
                [Rational('0.2'), Rational('0.3'), Rational('0.2'), Rational('0.1'), Rational('0.1'), Rational('0.1')],
                [Rational('0.1'), Rational('0.2'), Rational('0.3'), Rational('0.2'), Rational('0.1'), Rational('0.1')],
                [Rational('0.1'), Rational('0.1'), Rational('0.2'), Rational('0.3'), Rational('0.2'), Rational('0.1')],
                [Rational('0.1'), Rational('0.1'), Rational('0.1'), Rational('0.2'), Rational('0.3'), Rational('0.2')],
                [Rational('0.1'), Rational('0.1'), Rational('0.1'), Rational('0.1'), Rational('0.2'), Rational('0.4')]])
p_lin

Matrix([
[ 2/5,  1/5, 1/10, 1/10, 1/10, 1/10],
[ 1/5, 3/10,  1/5, 1/10, 1/10, 1/10],
[1/10,  1/5, 3/10,  1/5, 1/10, 1/10],
[1/10, 1/10,  1/5, 3/10,  1/5, 1/10],
[1/10, 1/10, 1/10,  1/5, 3/10,  1/5],
[1/10, 1/10, 1/10, 1/10,  1/5,  2/5]])

In [16]:
print(f'All row sum equals to 1 for nMC: {all([Add(*p.row(row_num)) == 1 for row_num in range(p.rows)])}')
print(f'All row sum equals to 1 for MC: {all([Add(*p_lin.row(row_num)) == 1 for row_num in range(p_lin.rows)])}')

All row sum equals to 1 for nMC: True
All row sum equals to 1 for MC: True


In [17]:
coupl_proc, eigenvalues, eigen_modulus = linear_coupled_process_spectral_radius(p_lin)

The coupled process state space size is 30

Pairs:
[[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [1, 0], [1, 2], [1, 3], [1, 4], [1, 5], [2, 0], [2, 1], [2, 3], [2, 4], [2, 5], [3, 0], [3, 1], [3, 2], [3, 4], [3, 5], [4, 0], [4, 1], [4, 2], [4, 3], [4, 5], [5, 0], [5, 1], [5, 2], [5, 3], [5, 4]]

Markov-Dobrushin 1-alpha:


2/5


Coupled process:


Matrix([
[1/10, 1/10,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
[   0,  1/5, 1/10,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
[   0, 3/40, 3/20, 3/40,    0,    0, 1/40, 1/20, 1/40,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
[   0,    0, 3/40, 3/20, 3/40,    0,    0, 1/40, 1/20, 1/40,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
[   0,    0,    0, 3/40, 9/40,    0,    0,    0, 1/40, 3/40,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
[   0,    0,    0,    0,    0, 1/10,    0,    0,    0,    0, 1/10,    0,    0,   


Eigen values:


[CRootOf(100*lambda**2 - 40*lambda + 1, 1),
 CRootOf(100*lambda**2 - 40*lambda + 1, 1),
 3/10,
 3/10,
 CRootOf(76800000*lambda**5 - 37120000*lambda**4 + 5680000*lambda**3 - 366400*lambda**2 + 10190*lambda - 101, 4),
 CRootOf(76800000*lambda**5 - 37120000*lambda**4 + 5680000*lambda**3 - 366400*lambda**2 + 10190*lambda - 101, 4),
 1/5,
 1/5,
 CRootOf(1600*lambda**2 - 360*lambda + 13, 1),
 CRootOf(1600*lambda**2 - 360*lambda + 13, 1),
 1/10,
 1/10,
 1/10,
 1/10,
 1/10,
 1/10,
 CRootOf(76800000*lambda**5 - 37120000*lambda**4 + 5680000*lambda**3 - 366400*lambda**2 + 10190*lambda - 101, 3),
 CRootOf(76800000*lambda**5 - 37120000*lambda**4 + 5680000*lambda**3 - 366400*lambda**2 + 10190*lambda - 101, 3),
 CRootOf(76800000*lambda**5 - 37120000*lambda**4 + 5680000*lambda**3 - 366400*lambda**2 + 10190*lambda - 101, 2),
 CRootOf(76800000*lambda**5 - 37120000*lambda**4 + 5680000*lambda**3 - 366400*lambda**2 + 10190*lambda - 101, 2),
 CRootOf(1600*lambda**2 - 360*lambda + 13, 0),
 CRootOf(1600*lambd


Eigen values modulus:


[CRootOf(100*lambda**2 - 40*lambda + 1, 1),
 CRootOf(100*lambda**2 - 40*lambda + 1, 1),
 3/10,
 3/10,
 CRootOf(76800000*lambda**5 - 37120000*lambda**4 + 5680000*lambda**3 - 366400*lambda**2 + 10190*lambda - 101, 4),
 CRootOf(76800000*lambda**5 - 37120000*lambda**4 + 5680000*lambda**3 - 366400*lambda**2 + 10190*lambda - 101, 4),
 1/5,
 1/5,
 CRootOf(1600*lambda**2 - 360*lambda + 13, 1),
 CRootOf(1600*lambda**2 - 360*lambda + 13, 1),
 1/10,
 1/10,
 1/10,
 1/10,
 1/10,
 1/10,
 CRootOf(76800000*lambda**5 - 37120000*lambda**4 + 5680000*lambda**3 - 366400*lambda**2 + 10190*lambda - 101, 3),
 CRootOf(76800000*lambda**5 - 37120000*lambda**4 + 5680000*lambda**3 - 366400*lambda**2 + 10190*lambda - 101, 3),
 CRootOf(76800000*lambda**5 - 37120000*lambda**4 + 5680000*lambda**3 - 366400*lambda**2 + 10190*lambda - 101, 2),
 CRootOf(76800000*lambda**5 - 37120000*lambda**4 + 5680000*lambda**3 - 366400*lambda**2 + 10190*lambda - 101, 2),
 CRootOf(1600*lambda**2 - 360*lambda + 13, 0),
 CRootOf(1600*lambd

Spectral radius $r$ is equal to the greatest eigenvalue modulus.

In [18]:
r = [N(eig_mod) for eig_mod in eigen_modulus][0]
r

0.373205080756888

In [19]:
linear_markov_dobrushin_one_minus_alpha(p_lin)

2/5

Let us calculate the values $1-\alpha_2, 1-\alpha_3, 1-\alpha_4, 1-\alpha_5$ compare them with $r^2, r^3, r^4, r^5$ respectively.

In [20]:
[N(linear_markov_dobrushin_one_minus_alpha(p_lin ** y)) for y in range(2,6)]

[0.160000000000000,
 0.0620000000000000,
 0.0236000000000000,
 0.00890000000000000]

In [21]:
[r ** y for y in range(2,6)]

[0.139282032302755,
 0.0519807621135332,
 0.0193994845223857,
 0.00723998618781895]

Therefore, $r \approx 0.3732$, $r^2 \approx 0.139282$, $r^3 \approx 0.05198$, $r^4 \approx 0.019399$, $r^5 \approx 0.007239986$, $1- \bar\alpha = 0.4$, $1- \bar\alpha_2 = 0.16$, $1 - \bar\alpha_3 = 0.062$, $1 - \bar\alpha_4 = 0.0236$, $1 - \bar\alpha_5 = 0.0089$.
We have $r < 1 - \bar\alpha = 1 - \alpha$, $r^2 < 1-\bar\alpha_2$, $r^3 < 1-\bar\alpha_3$, $r^4 < 1-\bar\alpha_4$, $r^5 < 1-\bar\alpha_5$. Again, the new assumption based on the spectral radius approach for linear MC and on nonlinear small perturbations is weaker that each of the MD conditions, at least, for $k=1, \ldots, 5$. 